# Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import math

import torch
from plaid.datasets import CATHShardedDataModule, CATHStructureDataModule
from plaid.esmfold.misc import batch_encode_sequences
from plaid.utils import LatentScaler
from plaid.losses.modules import SequenceAuxiliaryLoss, BackboneAuxiliaryLoss
from plaid.proteins import LatentToStructure, LatentToSequence

# Datasets

In [4]:
# shard_dir = "/data/lux70/data/rocklin/shards/"
# pdb_dir = "/data/lux70/data/rocklin/structures/" 
shard_dir = "/data/lux70/data/cath/shards/"
pdb_dir = "/data/lux70/data/cath/dompdb/"

embedder = "esmfold"
D = 1024 if embedder == "esmfold" else 320

dm = CATHShardedDataModule(
    storage_type="hdf5",
    shard_dir=shard_dir,
    embedder=embedder,
    seq_len=256,
    batch_size=64
)
# dm = CATHStructureDataModule(
#     shard_dir=shard_dir,
#     pdb_path_dir=pdb_dir,
#     embedder=embedder,
#     seq_len=256,
#     batch_size=16
# )
dm.setup()
train_dataloader = dm.train_dataloader()
val_dataloader = dm.val_dataloader()

In [5]:
latent_scaler = LatentScaler(lm_embedder_type=embedder)
# sequence_constructor = LatentToSequence()
# sequence_constructor.to("cuda")
# seq_loss = SequenceAuxiliaryLoss(sequence_constructor)

In [6]:
# from plaid.losses.modules import BackboneAuxiliaryLoss
# structure_constructor = LatentToStructure()
# structure_constructor.to('cuda')
# structure_loss_fn = BackboneAuxiliaryLoss(structure_constructor)

# Model

In [31]:
import torch
from torch import nn, einsum
import torch.nn.functional as F
from einops import rearrange, reduce, repeat

# helpers

def exists(val):
    return val is not None

def default(val, d):
    return val if exists(val) else d

def pad_to_multiple(tensor, multiple, dim = -1, value = 0):
    seq_len = tensor.shape[dim]
    m = seq_len / multiple
    if m.is_integer():
        return tensor
    remainder = math.ceil(m) * multiple - seq_len
    pad_offset = (0,) * (-1 - dim) * 2
    return F.pad(tensor, (*pad_offset, 0, remainder), value = value)

def cast_tuple(val, depth = 1):
    return val if isinstance(val, tuple) else ((val,) * depth)

# factory

def get_hourglass_transformer(
    dim,
    *,
    depth,
    shorten_factor,
    downproj_factor,
    attn_resampling,
    updown_sample_type,
    **kwargs
):
    assert isinstance(depth, int) or (isinstance(depth, tuple)  and len(depth) == 3), 'depth must be either an integer or a tuple of 3, indicating (pre_transformer_depth, <nested-hour-glass-config>, post_transformer_depth)'
    assert not (isinstance(depth, int) and shorten_factor), 'there does not need to be a shortening factor when only a single transformer block is indicated (depth of one integer value)'

    if isinstance(depth, int):
        return Transformer(dim = dim, depth = depth, **kwargs)

    return HourglassTransformer(dim = dim, depth = depth, shorten_factor = shorten_factor, downproj_factor = downproj_factor, attn_resampling = attn_resampling, updown_sample_type = updown_sample_type, **kwargs)

# up and down sample classes

class NaiveDownsample(nn.Module):
    def __init__(self, shorten_factor):
        super().__init__()
        self.shorten_factor = shorten_factor

    def forward(self, x):
        return reduce(x, 'b (n s) d -> b n d', 'mean', s = self.shorten_factor)

class NaiveUpsample(nn.Module):
    def __init__(self, shorten_factor):
        super().__init__()
        self.shorten_factor = shorten_factor

    def forward(self, x):
        return repeat(x, 'b n d -> b (n s) d', s = self.shorten_factor)

class LinearDownsample(nn.Module):
    def __init__(self, dim, shorten_factor):
        super().__init__()
        self.proj = nn.Linear(dim * shorten_factor, dim)
        self.shorten_factor = shorten_factor

    def forward(self, x):
        x = rearrange(x, 'b (n s) d -> b n (s d)', s = self.shorten_factor)
        return self.proj(x)

class LinearUpsample(nn.Module):
    def __init__(self, dim, shorten_factor):
        super().__init__()
        self.proj = nn.Linear(dim, dim * shorten_factor)
        self.shorten_factor = shorten_factor

    def forward(self, x):
        x = self.proj(x)
        return rearrange(x, 'b n (s d) -> b (n s) d', s = self.shorten_factor)

# classes

class PreNormLinearDownProjection(nn.Module):
    def __init__(self, dim, downproj_factor):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.proj = nn.Linear(dim, dim // downproj_factor)

    def forward(self, x):
        return self.proj(self.norm(x))

class PreNormLinearUpProjection(nn.Module):
    def __init__(self, dim, downproj_factor):
        super().__init__()
        self.norm = nn.LayerNorm(dim // downproj_factor)
        self.proj = nn.Linear(dim // downproj_factor, dim)

    def forward(self, x):
        return self.proj(self.norm(x))

class PreNormResidual(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs) + x

class Attention(nn.Module):
    def __init__(
        self,
        dim,
        heads = 8,
        dim_head = 64,
        dropout = 0.,
        causal = False
    ):
        super().__init__()
        self.heads = heads
        self.causal = causal
        self.scale = dim_head ** -0.5
        inner_dim = heads * dim_head

        self.to_q = nn.Linear(dim, inner_dim, bias = False)
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias = False)
        self.to_out = nn.Linear(inner_dim, dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, context = None, mask = None):
        h, device = self.heads, x.device
        kv_input = default(context, x)

        q, k, v = self.to_q(x), *self.to_kv(kv_input).chunk(2, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), (q, k, v))

        q = q * self.scale

        sim = einsum('b h i d, b h j d -> b h i j', q, k)
        mask_value = -torch.finfo(sim.dtype).max

        if exists(mask):
            mask = rearrange(mask, 'b j -> b () () j')
            sim = sim.masked_fill(~mask, mask_value)

        if self.causal:
            i, j = sim.shape[-2:]
            mask = torch.ones(i, j, device = device, dtype = torch.bool).triu_(j - i + 1)
            mask = rearrange(mask, 'i j -> () () i j')
            sim = sim.masked_fill(mask, mask_value)

        attn = sim.softmax(dim = -1)
        attn = self.dropout(attn)

        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)', h = h)
        return self.to_out(out)

def FeedForward(dim, mult = 4, dropout = 0.):
    return nn.Sequential(
        nn.Linear(dim, dim * mult),
        nn.GELU(),
        nn.Dropout(dropout),
        nn.Linear(dim * mult, dim)
    )

# transformer classes

class Transformer(nn.Module):
    def __init__(
        self,
        dim,
        *,
        depth,
        causal = False,
        heads = 8,
        dim_head = 64,
        attn_dropout = 0.,
        ff_mult = 4,
        ff_dropout = 0.,
        norm_out = False
    ):
        super().__init__()
        self.layers = nn.ModuleList([])

        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNormResidual(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = attn_dropout, causal = causal)),
                PreNormResidual(dim, FeedForward(dim, mult = ff_mult, dropout = ff_dropout))
            ]))

        self.norm = nn.LayerNorm(dim) if norm_out else nn.Identity()

    def forward(self, x, context = None, mask = None):
        for attn, ff in self.layers:
            x = attn(x, context = context, mask = mask)
            x = ff(x)

        return self.norm(x)

class HourglassTransformer(nn.Module):
    def __init__(
        self,
        dim,
        *,
        depth,
        shorten_factor = 2,
        downproj_factor = 2,
        attn_resampling = True,
        updown_sample_type = 'naive',
        heads = 8,
        dim_head = 64,
        causal = False,
        norm_out = False
    ):
        super().__init__()

        pre_layers_depth, valley_depth, post_layers_depth = depth

        # shorten factor
        if isinstance(shorten_factor, (tuple, list)):
            shorten_factor, *rest_shorten_factor = shorten_factor
        elif isinstance(valley_depth, int):
            shorten_factor, rest_shorten_factor = shorten_factor, None
        else:
            shorten_factor, rest_shorten_factor = shorten_factor, shorten_factor

        # downproj factor
        if isinstance(downproj_factor, (tuple, list)):
            downproj_factor, *rest_downproj_factor = downproj_factor
        elif isinstance(valley_depth, int):
            downproj_factor, rest_downproj_factor = downproj_factor, None
        else:
            downproj_factor, rest_downproj_factor = downproj_factor, downproj_factor

        transformer_kwargs = dict(
            heads = heads,
            dim_head = dim_head
        )

        self.causal = causal
        self.shorten_factor = shorten_factor
        self.downproj_factor = downproj_factor

        if updown_sample_type == 'naive':
            self.downsample = NaiveDownsample(shorten_factor)
            self.upsample   = NaiveUpsample(shorten_factor)
        elif updown_sample_type == 'linear':
            self.downsample = LinearDownsample(dim, shorten_factor)
            self.upsample   = LinearUpsample(dim, shorten_factor)
        else:
            raise ValueError(f'unknown updown_sample_type keyword value - must be either naive or linear for now')

        self.down_projection = PreNormLinearDownProjection(dim, downproj_factor)
        self.up_projection = PreNormLinearUpProjection(dim, downproj_factor)
        
        self.valley_transformer = get_hourglass_transformer(
            dim = dim // downproj_factor,
            shorten_factor = rest_shorten_factor,
            downproj_factor = rest_downproj_factor,
            depth = valley_depth,
            attn_resampling = attn_resampling,
            updown_sample_type = updown_sample_type,
            causal = causal,
            **transformer_kwargs
        )

        self.attn_resampling_context_downproj = PreNormLinearDownProjection(dim, downproj_factor) if attn_resampling else None
        self.attn_resampling_context_upproj = PreNormLinearUpProjection(dim, downproj_factor) if attn_resampling else None
        self.attn_resampling_pre_valley = Transformer(dim = dim // downproj_factor, depth = 1, **transformer_kwargs) if attn_resampling else None
        self.attn_resampling_post_valley = Transformer(dim = dim, depth = 1, **transformer_kwargs) if attn_resampling else None

        self.pre_transformer = Transformer(dim = dim, depth = pre_layers_depth, causal = causal, **transformer_kwargs)
        self.post_transformer = Transformer(dim = dim, depth = post_layers_depth, causal = causal, **transformer_kwargs)
        self.norm_out = nn.LayerNorm(dim) if norm_out else nn.Identity()

    def forward(self, x, mask = None):
        # b : batch, n : sequence length, d : feature dimension, s : shortening factor
        print('inp', x.shape)
        s, b, n = self.shorten_factor, *x.shape[:2]
        
        # top half of hourglass, pre-transformer layers
        x = self.pre_transformer(x, mask = mask)
        print('after pre transformer', x.shape)
        
        # pad to multiple of shortening factor, in preparation for pooling
        x = pad_to_multiple(x, s, dim = -2)

        if exists(mask):
            padded_mask = pad_to_multiple(mask, s, dim = -1, value = False)
        print('after pad to multiple', x.shape)

        # save the residual, and for "attention resampling" at downsample and upsample
        x_residual = x.clone()

        # if autoregressive, do the shift by shortening factor minus one
        if self.causal:
            shift = s - 1
            x = F.pad(x, (0, 0, shift, -shift), value = 0.)

            if exists(mask):
                padded_mask = F.pad(padded_mask, (shift, -shift), value = False)
        print('after potentially short', x.shape)

        # naive average pool
        downsampled = self.downsample(x)
        print('after downsample', downsampled.shape)
        if exists(mask):
            downsampled_mask = reduce(padded_mask, 'b (n s) -> b n', 'sum', s = s) > 0
        else:
            downsampled_mask = None

        # also possibly reduce along dim=-1
        downsampled = self.down_projection(downsampled)
        print('after downproj', downsampled.shape)

        # pre-valley "attention resampling" - they have the pooled token in each bucket attend to the tokens pre-pooled
        if exists(self.attn_resampling_pre_valley):
            if exists(mask):
                attn_resampling_mask = rearrange(padded_mask, 'b (n s) -> (b n) s', s = s)
            else:
                attn_resampling_mask = None
            print('after pre valley attention resampling', x.shape)
            downsampled = self.attn_resampling_pre_valley(
                rearrange(downsampled, 'b n d -> (b n) () d'),
                rearrange(self.attn_resampling_context_downproj(x), 'b (n s) d -> (b n) s d', s = s),
                mask = attn_resampling_mask
            )

            downsampled = rearrange(downsampled, '(b n) () d -> b n d', b = b)
            print('after pre valley resampling rearrange', downsampled.shape)
            
        # the "valley" - either a regular transformer or another hourglass
        x = self.valley_transformer(downsampled, mask = downsampled_mask)
        print('after valley output', x.shape)
        valley_out = x.clone()

        # naive repeat upsample
        x = self.upsample(x)
        print('after upsample', x.shape)
        x = self.up_projection(x)
        print('after up proj', x.shape)
        
        # add the residual
        print("x_residual shape", x_residual.shape)
        x = x + x_residual
        
        # post-valley "attention resampling"
        if exists(self.attn_resampling_post_valley):
            x = self.attn_resampling_post_valley(
                rearrange(x, 'b (n s) d -> (b n) s d', s = s),
                rearrange(self.attn_resampling_context_upproj(valley_out), 'b n d -> (b n) () d')
            )

            x = rearrange(x, '(b n) s d -> b (n s) d', b = b)

        print('after post valley attention resampling', x.shape)

        # bring sequence back to original length, if it were padded for pooling
        x = x[:, :n]
        print('after maybe remove pad', x.shape)
        
        # post-valley transformers
        x = self.post_transformer(x, mask = mask)
        print('after post valley transformer', x.shape)
        return self.norm_out(x)

transformer = get_hourglass_transformer(
    dim = D,                     # feature dimension
    heads = 8,                      # attention heads
    dim_head = 64,                  # dimension per attention head
    shorten_factor = 2,             # shortening factor
    downproj_factor = 2,
    depth = (4, 2, 4),              # tuple of 3, standing for pre-transformer-layers, valley-transformer-layers (after downsample), post-transformer-layers (after upsample) - the valley transformer layers can be yet another nested tuple, in which case it will shorten again recursively
    attn_resampling = True,
    updown_sample_type = "naive",
    causal = True,
    norm_out = True
)

# Train

## Sample Batch

In [17]:
batch = next(iter(train_dataloader))

sequences = batch[1]
tokens, mask, _, _, _ = batch_encode_sequences(sequences)

x = batch[0]
if embedder != "esmfold":
    x = x[:, 1:-1, :]
mask = mask.bool()
print(x.shape)
print(mask.shape)



torch.Size([64, 256, 1024])
torch.Size([64, 256])
tensor(1.3661, device='cuda:0') tensor(72.3197, device='cuda:0')
tensor(-0.0006, device='cuda:0') tensor(0.2281, device='cuda:0')


In [33]:
transformer = get_hourglass_transformer(
    dim = D,                     # feature dimension
    heads = 8,                      # attention heads
    dim_head = 64,                  # dimension per attention head
    shorten_factor = (2, 2),             # shortening factor
    downproj_factor = (2, 2),
    depth = (4, (4, 2, 4), 4),              # tuple of 3, standing for pre-transformer-layers, valley-transformer-layers (after downsample), post-transformer-layers (after upsample) - the valley transformer layers can be yet another nested tuple, in which case it will shorten again recursively
    attn_resampling = True,
    updown_sample_type = "naive",
    causal = True,
    norm_out = True
)

device = "cuda"
transformer = transformer.to(device)
x, mask = x.to(device), mask.to(device)
print(x.mean(), x.std())

x = latent_scaler.scale(x)
print(x.mean(), x.std())

output = transformer(x, mask)

tensor(0.0034, device='cuda:0') tensor(0.1472, device='cuda:0')
tensor(0.0034, device='cuda:0') tensor(0.1472, device='cuda:0')
inp torch.Size([64, 256, 1024])
after pre transformer torch.Size([64, 256, 1024])
after pad to multiple torch.Size([64, 256, 1024])
after potentially short torch.Size([64, 256, 1024])
after downsample torch.Size([64, 128, 1024])
after downproj torch.Size([64, 128, 512])
after pre valley attention resampling torch.Size([64, 256, 1024])
after pre valley resampling rearrange torch.Size([64, 128, 512])
inp torch.Size([64, 128, 512])
after pre transformer torch.Size([64, 128, 512])
after pad to multiple torch.Size([64, 128, 512])
after potentially short torch.Size([64, 128, 512])
after downsample torch.Size([64, 64, 512])
after downproj torch.Size([64, 64, 256])
after pre valley attention resampling torch.Size([64, 128, 512])
after pre valley resampling rearrange torch.Size([64, 64, 256])
after valley output torch.Size([64, 64, 256])
after upsample torch.Size([64, 

In [14]:
# structure_loss_fn(output, batch[-1], batch[1])

Generating structure from latents


(tensor(0.9506, device='cuda:0'), {'backbone_loss': 0.9506101012229919})

### Reconstruct Sequence

In [ ]:
unnormalized_x_recons = latent_scaler.unscale(output)
sequence_constructor = LatentToSequence()
probs, _, strs = sequence_constructor.to_sequence(unnormalized_x_recons, mask, return_logits=False)

In [ ]:
print(unnormalized_x_recons.mean(), unnormalized_x_recons.std())

In [ ]:
strs[:10]  # random strings

## Train Loop

In [ ]:
from tqdm.notebook import tqdm, trange
import torch.nn.functional as F
from torch.optim import AdamW

optimizer = AdamW(transformer.parameters(), lr=1e-4)
n_epochs = 100

losses = []

for epoch in trange(n_epochs): 
    for i, batch in enumerate(train_dataloader):
        sequences = batch[1]
        tokens, mask, _, _, _ = batch_encode_sequences(sequences)

        x = batch[0]
        if embedder != "esmfold":
            x = x[:, 1:-1, :]
        else:
            x = latent_scaler.scale(x)
        mask = mask.bool()
        x, mask = x.to(device), mask.to(device)
        
        output = transformer(x, mask)
        recons_loss = F.mse_loss(x, output)
        if i % 50 == 0:
            print(loss.item()) 

        unnormalized_x_recons = latent_scaler.unscale(output)
        
        logits, _, strs = sequence_constructor.to_sequence(unnormalized_x_recons, mask, return_logits=True)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses)